[![Fixel Algorithms](https://i.imgur.com/AqKHVZ0.png)](https://fixelalgorithms.gitlab.io)

# AI Program

## Deep Learning - Computer Vision - Shapely Values for Deep Learning

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 21/03/2025 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0099DeepLearningObjectDetection.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
import shap

# Deep Learning
import torch
import torch.nn            as nn
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer
from torch.optim.lr_scheduler import LRScheduler
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchinfo
from torchmetrics.classification import MulticlassAccuracy
import torchvision
from torchvision.transforms import v2 as TorchVisionTrns

# Miscellaneous
from enum import auto, Enum, unique
import math
import os
from platform import python_version
import random
import time

# Typing
from typing import Callable, Dict, Generator, List, Optional, Self, Set, Tuple, Union

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

# Improve performance by benchmarking
torch.backends.cudnn.benchmark = True

# Reproducibility (Per PyTorch Version on the same device)
# torch.manual_seed(seedNum)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark     = False #<! Makes things slower


In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

DATA_SET_FILE_NAME      = 'archive.zip'
DATA_SET_FOLDER_NAME    = 'IntelImgCls'

D_CLASSES   = {ii: str(ii) for ii in range(10)}
L_CLASSES   = [str(ii) for ii in range(10)]
TU_IMG_SIZE = (28, 28, 1)

PROJECT_NAME     = 'FixelCourses'
DATA_FOLDER_PATH = 'DataSets'
BASE_FOLDER      = os.getcwd()[:(len(os.getcwd()) - (os.getcwd()[::-1].lower().find(PROJECT_NAME.lower()[::-1])))]

TENSOR_BOARD_BASE   = 'TB'


In [ ]:
# Download Auxiliary Modules for Google Colab
if runInGoogleColab:
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DataManipulation.py
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DataVisualization.py
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DeepLearningPyTorch.py

In [ ]:
# Courses Packages

from DataVisualization import PlotLabelsHistogram, PlotMnistImages
from DeepLearningPyTorch import TrainModel


In [ ]:
# General Auxiliary Functions

def TensorImageNumpy( tZ: torch.Tensor ) -> np.ndarray:
    """
    Converts a PyTorch Tensor to a Numpy Array.
    """
    mZ = tZ.squeeze()
    mX = mZ.detach().cpu().numpy()

    return mX


## Shapely Values

![](https://i.postimg.cc/BbkcrzRM/shap-headeraa.png)
<!-- ![](https://i.imgur.com/oCqTLBk.png) -->

The [SHAP](https://github.com/shap/shap) (SHapley Additive exPlanations) package allows to create explanation per **processed sample**.  
It is based on a game theoretic approach to explain the output of **any** _Machine Learning Model_.



* <font color='brown'>(**#**)</font> The concept of the Shapely Values is to measure the marginal contribution of each feature.   
  The concept is to look at any combination of the features 
* <font color='brown'>(**#**)</font> The values can be used for _Feature Importance_ for _Feature Engineering_ during the training process.
* <font color='brown'>(**#**)</font> Features pushing the prediction higher are shown in red, those pushing the prediction lower are in blue.
* <font color='brown'>(**#**)</font> Resources on Shapely Values:
  - [Understanding SHAP Values: A Panoramic Guide](https://scribe.rip/132e817c01f0).
  - [Aidan Cooper - Explaining Machine Learning Models: A Non Technical Guide to Interpreting SHAP Analyses](https://www.aidancooper.co.uk/a-non-technical-guide-to-interpreting-shap-analyses).
  - [Problems with Shapley Value Based Explanations as Feature Importance Measures](https://hdc.cs.arizona.edu/papers/html/icml_2020_shapley.html).
  - [Interpretable Machine Learning - Shapley Values](https://christophm.github.io/interpretable-ml-book/shapley.html).
  - [Reddit - Shapely Values Discussion](https://www.reddit.com/r/datascience/comments/w5d3zg).

In [ ]:
# Parameters

# Data
numSamplesTrain = 30_000
numSamplesVal   = 10_000
numCls          = len(L_CLASSES) #<! Number of classes
maxObj          = 3

dataFolder = os.path.join(BASE_FOLDER, DATA_FOLDER_PATH)

# Model
latDim     = 2
latDimFctr = 8 #<! Linear Layer

# Training
batchSize   = 512
numWorkers  = 2 #<! Number of workers
numEpochs   = 5

# Visualization
numImg = 3


## Generate / Load Data

The examples uses the MNIST data set.

In [ ]:
# Loader Transform

oTrns = TorchVisionTrns.Compose([
    TorchVisionTrns.ToImage(),
    TorchVisionTrns.ToDtype(torch.float, scale = True),
])

In [ ]:
# Data Set

dsTrain = torchvision.datasets.MNIST(root = dataFolder, train = True,  transform = oTrns, download = True)
dsVal   = torchvision.datasets.MNIST(root = dataFolder, train = False, transform = oTrns, download = True)

print(f'The training data set RAW data shape: {dsTrain.data.shape}')
print(f'The validation data set RAW data shape: {dsVal.data.shape}')

In [ ]:
# Element of the Data Set / Data Sample

tX, valY = dsTrain[0]

print(f'The features shape: {tX.shape}')
print(f'The label         : {valY}')

### Plot the Data

In [ ]:
# Plot the Data

mX = np.reshape(dsTrain.data.numpy(), (dsTrain.data.shape[0], -1))
vY = dsTrain.targets.numpy()

hF = PlotMnistImages(mX, vY, 3, 3)


In [ ]:
# Histogram of Labels

hA = PlotLabelsHistogram(vY, lClass = L_CLASSES)
plt.show()

## Train Classifier

A simple CNN model.

* <font color='brown'>(**#**)</font> The SHAP package does not support all operations of PyTorch.  
  See [supported operations of SHAP Deep Explainer](https://github.com/shap/shap/blob/0d1ae5a62c9e0cafc2368fddd1b76e670cf44cb5/shap/explainers/_deep/deep_pytorch.py#L375) (As of `2025_03`).

In [ ]:
oClsModel = nn.Sequential(
    nn.Identity(),
    
    nn.Conv2d(in_channels = 1, out_channels = 30, kernel_size = 7, bias = False),
    nn.MaxPool2d(kernel_size = 2),
    nn.BatchNorm2d(num_features = 30),
    nn.ReLU(),
    
    nn.Conv2d(in_channels = 30, out_channels = 60, kernel_size = 5, bias = False),
    nn.MaxPool2d(kernel_size = 2),
    nn.BatchNorm2d(num_features = 60),
    nn.ReLU(),
            
    nn.Conv2d(in_channels = 60,  out_channels = 120, kernel_size = 3, bias = False),
    nn.BatchNorm2d(num_features = 120),
    nn.ReLU(),
    
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(120, 10),
)

torchinfo.summary(oClsModel, (256, *(TU_IMG_SIZE[::-1])), col_names = ['kernel_size', 'output_size', 'num_params'], device = 'cpu', row_settings = ['depth', 'var_names'])


In [ ]:
# Data Loader

dlTrain = torch.utils.data.DataLoader(dsTrain, shuffle = True, batch_size = 1 * batchSize, num_workers = numWorkers, drop_last = True, persistent_workers = True)
dlVal   = torch.utils.data.DataLoader(dsVal, shuffle = False, batch_size = 2 * batchSize, num_workers = numWorkers, persistent_workers = True)

In [ ]:
# Check GPU Availability

runDevice   = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') #<! The 1st CUDA device

In [ ]:
# Loss and Score
hL = nn.CrossEntropyLoss()
hS = MulticlassAccuracy(num_classes = 10, average = 'micro')
hL = hL.to(runDevice) #<! Not required!
hS = hS.to(runDevice)

In [ ]:
oClsModel = oClsModel.to(runDevice) #<! Transfer model to device
oOpt = torch.optim.AdamW(oClsModel.parameters(), lr = 6e-4, betas = (0.9, 0.99), weight_decay = 1e-3) #<! Define optimizer
oRunModel, lTrainLoss, lTrainScore, lValLoss, lValScore, lLearnRate = TrainModel(oClsModel, dlTrain, dlVal, oOpt, numEpochs, hL, hS)

In [ ]:
# Plot Training Phase

hF, vHa = plt.subplots(nrows = 1, ncols = 3, figsize = (15, 5))
vHa = np.ravel(vHa)

hA = vHa[0]
hA.plot(lTrainLoss, lw = 2, label = 'Train')
hA.plot(lValLoss, lw = 2, label = 'Validation')
hA.set_title(f'Classification Loss')
hA.set_xlabel('Epoch')
hA.set_ylabel('Loss')
hA.legend()

hA = vHa[1]
hA.plot(lTrainScore, lw = 2, label = 'Train')
hA.plot(lValScore, lw = 2, label = 'Validation')
hA.set_title('Classification Score')
hA.set_xlabel('Epoch')
hA.set_ylabel('Score')
hA.legend()

hA = vHa[2]
hA.plot(lLearnRate, lw = 2)
hA.set_title('Learn Rate Scheduler')
hA.set_xlabel('Epoch')
hA.set_ylabel('Learn Rate');

### Expandability of Predictions

This section use the [SHAP](https://github.com/shap/shap) package for expandability of the predictions.

In [ ]:
# Convert to CPU
# The SHAP analyzer requires the model to be on the CPU.

oClsModel = oClsModel.to('cpu')

In [ ]:
# Get Baseline

tX, vY = next(iter(dlVal))

tB = tX[:1000] #<! Baseline
tP = tX[1000:1009] #<! Predictions

oShapExp = shap.DeepExplainer(oClsModel, tB)
tShapVal = oShapExp.shap_values(tP)

In [ ]:
# Convert to Numpy
lShapVal = list(np.transpose(tShapVal, (4, 0, 2, 3, 1)))
tPShow   = np.swapaxes(np.swapaxes(tP.numpy(), 1, -1), 1, 2)

### SHAP Values
The DL Explainer shows the contribution of each average feature to the current decision.  
Hence each column is class and the values are contributions to the class values (The classifier takes the max over each row).

* <font color='brown'>(**#**)</font> See [Partition Explainer example](https://shap.readthedocs.io/en/latest/example_notebooks/image_examples/image_classification/Explain%20MobilenetV2%20using%20the%20Partition%20explainer%20%28PyTorch%29.html). It uses "blurring" effect to see the contribution of each partition.

In [ ]:
# Plot the Values per Class
shap.image_plot(lShapVal, -tPShow)